<a target="_blank" href="https://colab.research.google.com/github/msako82/DiffPharma/blob/main/colab/DiffPharma_generate.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Change runtime type to T4 GPU

In [ ]:
#@title you need to check the numpy version = 1.23.5. Don't restart session
!pip uninstall -y numpy
!pip install numpy==1.23.5

import numpy
print("NumPy version:", numpy.__version__)

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [ ]:
#@title Install module list

errors ={}
commands = [
    "conda install openbabel -c conda-forge",
    "conda install conda-forge/label/cf202003::oddt",
    "pip install scipy==1.9.3 scikit-learn==1.1.3 --no-cache-dir --force-reinstall",
    "pip install torch==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu118",
    "pip install torch torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118",
    "pip install pytorch-lightning==1.8.4",
    "pip install rdkit==2022.9.2",
    "pip install biopython==1.79",
    "pip install imageio==2.21.2",
    "pip install pyg-lib torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html",
    "pip install networkx==2.8.6",
    "pip install py3Dmol",
    "pip install wandb==0.13.1",
    "pip install seaborn",
]

In [ ]:
#@title Install module (about 5 minutes)
import os

if not os.path.isfile("/content/READY"):
  for cmd in commands:
    # os.system(cmd)
    with os.popen(cmd) as f:
      out = f.read()
      status = f.close()

    if status is not None:
      errors[cmd] = out
      print(f"\n\nAn error occurred while running '{cmd}'\n")
      print("Status:\t", status)
      print("Message:\t", out)

In [4]:
%env MPLBACKEND=Agg

env: MPLBACKEND=Agg


In [ ]:
#@title download git
!git clone https://github.com/sekijima-lab/DiffPharma.git

Cloning into 'DiffPharma01'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 6), reused 42 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 3.75 MiB | 43.68 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
#@title Change Directory
cd DiffPharma/

/content/DiffPharma01


In [ ]:
#@title Data construction of 10 molecules
!python test_single_14gs.py --sdf example/14gs.sdf --pdb example/14gs.pdb --checkpoint DiffPharma_best.ckpt --outdir tmp_out

num_nodes_lig: tensor([13, 35, 12,  9, 15, 18], device='cuda:0')
<class 'equivariant_diffusion.conditional_model.ConditionalDDPM'>
/usr/local/lib/python3.11/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
[13:04:40] Molecule does not have explicit Hs. Consider calling AddHs()
[13:04:40] Molecule does not have explicit Hs. Consider calling AddHs()
/content/DiffPharma01/analysis/molecule_builder.py:224: UserWarning: Maximum number of FF iterations reached. Returning molecule after 200 relaxation steps.
  warnings.warn(f'Maximum number of FF iterations reached. '
[13:04:40] Molecule does not have explicit Hs. Consider calling AddHs()
[13:04:40] Molecule does not have explicit Hs. Consider calling AddHs()
[13:04:40] Molecule does not have explicit Hs. Consider calling AddHs()
[13:04:40] Molecule does not have

In [ ]:
from rdkit import Chem
import py3Dmol

In [11]:
#@title Ooen PDB for Visualization
with open("example/14gs.pdb") as f:
    system = "".join([x for x in f])

In [ ]:
#@title Read Generated Molecules
suppl = Chem.SDMolSupplier('tmp_out/processed/14gs_gen.sdf')
out_sdf = [mol for mol in suppl if mol is not None]
print(f"number of molecules: {len(out_sdf)}")

In [ ]:
#@title Visualization
viewA = py3Dmol.view(width=400, height=300)
id = 0  # select number of molecules (0-9)
mol_block = Chem.MolToMolBlock(out_sdf[id])
viewA.addModel(mol_block, 'sdf')
viewA.setStyle({'stick': {}})
viewA.addModelsAsFrames(system)
viewA.addSurface(py3Dmol.VDW,{'opacity':0.9,'color':'white'})

viewA.zoomTo()
viewA.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.